In [66]:
import pandas as pd
import numpy as np

In [109]:
#BASE = 'afastamentos'
BASE = 'cadastro'
df = pd.read_csv(BASE + '-nomes-sexo.csv', index_col=0)

In [110]:
# remover valores nulos
df.drop(df[df.index.isnull()].index, inplace=True)

In [111]:
df.rename(columns={'sexo': 'SEXO'}, inplace=True)

In [112]:
df.head(10)

,SEXO
PNOME,
MARIA,F
JOSE,M
ANA,F
JOAO,M
CARLOS,M
PAULO,M
ANTONIO,M
LUIZ,M
FRANCISCO,M


In [113]:
df['SEXO'] = df['SEXO'].map({'F': 1, 'M': 2, 'X': 0}).astype('uint8')

In [114]:
df['REV'] = df.index
df['REV'] = df['REV'].apply(lambda x: x[::-1])

In [115]:
df.head()

,SEXO,REV
PNOME,,
MARIA,1,AIRAM
JOSE,2,ESOJ
ANA,1,ANA
JOAO,2,OAOJ
CARLOS,2,SOLRAC


In [116]:
df[['SEXO', 'REV']].groupby(by=['SEXO']).count().rename({'REV': 'QTDE'}, axis=1)

,QTDE
SEXO,
0,7820
1,21891
2,19259


In [117]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

for i in range(MAIOR_QTDE_LETRAS):
    df['L' + str(i)] = df['REV'].apply(
        lambda x: ord(x[i]) - 64 if len(x) > i else 0
    ).astype('uint8')

In [118]:
#df.dtypes

In [119]:
df.head(10)

,SEXO,REV,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,
MARIA,1,AIRAM,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,2,ESOJ,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,1,ANA,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,2,OAOJ,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,2,SOLRAC,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0
PAULO,2,OLUAP,15,12,21,1,16,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,2,OINOTNA,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
LUIZ,2,ZIUL,26,9,21,12,0,0,0,0,0,0,0,0,0,0,0,0
FRANCISCO,2,OCSICNARF,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0


In [120]:
#df.describe()

### Decomposição com PCA

In [121]:
from sklearn.decomposition import PCA

X_reduced = PCA(n_components=4).fit_transform(df.drop(['SEXO', 'REV'], axis=1))
X_reduced[:5]

array([[ -7.76160475,  -2.07470277, -13.8857795 ,  -8.75521502],
       [-11.19034433,  -9.16743323,  -1.61160848,  -0.87658742],
       [ -8.66620096, -12.77906996,  -5.41728336,  -1.16372348],
       [-14.56344849,   5.52587062,   1.97334888,   2.65486367],
       [-13.56301814,  -0.07901843,   7.75461128,   6.89986587]])

In [122]:
X = pd.DataFrame(X_reduced, index=df.index)
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (48970, 4) (48970,)


### Uso dos dados originais

In [123]:
# definir dados de entrada

X = df.drop(['SEXO', 'REV'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (48970, 16) (48970,)


In [83]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
y.head()

PNOME
MARIA        1
JOSE         2
ANTONIO      2
FRANCISCO    2
JOAO         2
Name: SEXO, dtype: uint8

In [85]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [86]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [87]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [88]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [89]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    random_state=42, multi_class='auto', 
    max_iter=1000, solver='liblinear', C=1.0) # 85.18
evaluate_model('LR', model)

params = dict(
    solver=['liblinear','lbfgs'], 
    C=np.logspace(-3,3,7)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 85.27 (+/- 0.70) [ 2587 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished


In [90]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(
    C=np.logspace(-4,3,8)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 85.16 (+/- 0.68) [  740 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


In [91]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 87.45
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(n_neighbors=11) 
Cross-Validation Score: 87.64 (+/- 0.52) [ 2538 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s finished


In [92]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(
    solver=['svd','lsqr','eigen']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 85.18 (+/- 0.64) [  220 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [93]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 10.44 (+/- 0.71) [  123 ms]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [94]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=1.0) # 67.62
evaluate_model('GNB', model)

params = dict(
    var_smoothing=np.logspace(0,-9,num=10)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(var_smoothing=1.0) 
Cross-Validation Score: 85.29 (+/- 0.68) [  117 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [95]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

#params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 44.21 (+/- 0.56) [  118 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


(44.21145213832606, 0.5612128048081376, 118)

In [96]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    random_state=42, criterion='entropy', max_depth=7) # 96.37
evaluate_model('DT', model)

params = dict(
    criterion=['gini','entropy'],
    max_depth=[3,5,7,11,13,17,19]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=42) 
Cross-Validation Score: 94.13 (+/- 0.30) [  173 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [97]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42, max_features='sqrt', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(
    n_estimators=[10,50,100,500], 
    max_features=['auto','sqrt','log2']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestClassifier(max_features='sqrt', n_estimators=500, random_state=42) 
Cross-Validation Score: 93.42 (+/- 0.62) [19672 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.7s finished


In [98]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), 
    n_estimators=11) # 96.41
evaluate_model('ABDT', model)

params = dict(
    n_estimators=[1,3,5,7,9,11,13]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=11) 
Cross-Validation Score: 93.85 (+/- 1.23) [ 1360 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


In [99]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=350, tol=0.01) # 77.02
evaluate_model('SGD', model)

params = dict(
    max_iter=[100, 200, 350, 500, 1000], 
    tol=[0.01, 0.1, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(max_iter=350, random_state=42, tol=0.01) 
Cross-Validation Score: 78.82 (+/- 3.98) [ 2006 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


In [100]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.1) # 75.58
evaluate_model('PCT', model)

params = dict(
    max_iter=[100, 200, 350, 500, 750, 1000], 
    tol=[0.1, 0.01, 0.001]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(max_iter=500, random_state=42, tol=0.1) 
Cross-Validation Score: 77.55 (+/- 2.21) [  310 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [101]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(
    max_depth=3, min_child_weight=3, gamma=0.4, 
    subsample=0.7, colsample_bytree=0.8, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, gamma=0.4,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.7, tree_method=None,
              validate_parameters=None, verbosity=None) 
Cross-Validation Score: 94.75 (+/- 0.68) [ 9388 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.4s finished


In [102]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(
    num_leaves=20, max_depth=3, 
    learning_rate=0.1, feature_fraction=1.0) # 97.51
evaluate_model('LGBM', model)

params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LGBMClassifier(feature_fraction=1.0, max_depth=3, num_leaves=20) 
Cross-Validation Score: 94.40 (+/- 0.49) [ 3188 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.2s finished


In [103]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(
    silent=True, iterations=250,
    learning_rate=0.05, depth=7) # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(
    iterations=[10,50,100,250,500,1000],
    learning_rate=[0.01,0.05,0.1,0.5],
    depth=range(1,11,2)
) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 94.49 (+/- 0.49) [11935 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.9s finished


In [104]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [105]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
12,XGB,94.745935,0.681502,9388,"XGBClassifier(base_score=None, booster=None, c..."
14,CB,94.488944,0.485484,11935,<catboost.core.CatBoostClassifier object at 0x...
13,LGBM,94.396410,0.490468,3188,"LGBMClassifier(feature_fraction=1.0, max_depth..."
7,DT,94.129137,0.302274,173,"DecisionTreeClassifier(criterion='entropy', ma..."
9,ABDT,93.851719,1.232554,1360,AdaBoostClassifier(base_estimator=DecisionTree...
8,RF,93.419641,0.615820,19672,"RandomForestClassifier(max_features='sqrt', n_..."
2,KNN,87.641339,0.519637,2538,KNeighborsClassifier(n_neighbors=11)
5,GNB,85.286784,0.682386,117,GaussianNB(var_smoothing=1.0)
0,LR,85.266208,0.696586,2587,"LogisticRegression(max_iter=1000, random_state..."
3,LDA,85.183983,0.644626,220,LinearDiscriminantAnalysis()


In [106]:
model = models['XGB'][0]
model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, gamma=0.4,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.7, tree_method=None,
              validate_parameters=None, verbosity=None)

In [108]:
# treinar o melhor modelo com todos os dados
model.fit(X, y)

[11:58:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=None, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [124]:
y_pred = model.predict(X)
y_pred

array([1, 1, 1, ..., 2, 2, 1], dtype=uint8)

In [125]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

,REAL,PREV
PNOME,,
MARIA,F,F
JOSE,M,F
ANA,F,F
JOAO,M,M
CARLOS,M,M


In [126]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

0.9459260771901163

In [127]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

array([[ 5851,  1076,   893],
       [  152, 21569,   170],
       [  296,    61, 18902]])

In [128]:
dados[dados['REAL'] != dados['PREV']].head(20)

,REAL,PREV
PNOME,,
JOSE,M,F
LUIZ,M,F
JORGE,M,X
GUILHERME,M,X
RAQUEL,F,M
THAIS,F,X
BEATRIZ,F,X
LILIAN,F,M
SOLANGE,F,X
